# Experimental Pandas querying for Dashboard

In [28]:
from os import environ

from boto3 import client
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
from psycopg2 import connect, extras
from psycopg2.extensions import connection

load_dotenv()


SELECT_ALL_QUERY = """SELECT prices.price_id, prices.updated_at, prices.price, products.product_id, products.product_name, products.product_url, products.website_name, products.image_url, products.product_availability, users.user_id, users.email, users.first_name, users.last_name, subscriptions.subscription_id
                FROM prices
                JOIN products ON products.product_id = prices.product_id
                JOIN subscriptions ON subscriptions.product_id = products.product_id
                JOIN users ON users.user_id = subscriptions.user_id
                ORDER BY product_id, updated_at DESC;"""

COLUMNS = {"price_id": "Price ID", "updated_at": "Updated At", "price": "Price", "product_id": "Product ID", "product_name": "Product Name", "product_url": "Product URL", "image_url": "Image URL",
           "product_availability": "Product Availability", "website_name": "Website Name", "user_id": "User ID", "first_name": "User FirstName", "last_name": "User LastName", "email": "User Email", "subscription_id": "Subscription ID"}

## Making the Database connection

In [29]:
def get_database_connection() -> connection:
    """
    Return a connection our database.
    """
    try:
        return connect(
            user=environ["DB_USER"],
            password=environ["DB_PASSWORD"],
            host=environ["DB_HOST"],
            port=environ["DB_PORT"],
            database=environ["DB_NAME"]
        )
    except ConnectionError as error:
        return error


conn = get_database_connection()

### The Entire Merged Table

In [30]:
def load_all_database_info(db_conn: connection) -> pd.DataFrame:
    """Extract all data from the database."""

    with db_conn.cursor(cursor_factory=extras.RealDictCursor) as cur:

        cur.execute(SELECT_ALL_QUERY)

        result = cur.fetchall()

        return pd.DataFrame(result).rename(columns=COLUMNS)
    

all_info = load_all_database_info(conn)

all_info.head(20)

,Price ID,Updated At,Price,Product ID,Product Name,Product URL,Website Name,Image URL,Product Availability,User ID,User Email,User FirstName,User LastName,Subscription ID
0,228,2024-01-05 17:19:05.936439,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
1,215,2024-01-05 17:11:54.939769,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
2,204,2024-01-05 17:10:07.228039,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
3,193,2024-01-05 17:09:35.975965,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
4,182,2024-01-05 17:08:08.702625,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
5,171,2024-01-05 17:08:03.462702,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
6,160,2024-01-05 17:07:58.464333,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
7,149,2024-01-05 17:07:47.847767,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
8,138,2024-01-05 17:04:13.958043,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
9,127,2024-01-05 17:04:04.511574,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1


### Finding All Information of just 1 user

In [31]:
user_id_1_df = all_info[all_info['User ID'] == 1]

user_id_1_df

,Price ID,Updated At,Price,Product ID,Product Name,Product URL,Website Name,Image URL,Product Availability,User ID,User Email,User FirstName,User LastName,Subscription ID
0,228,2024-01-05 17:19:05.936439,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
1,215,2024-01-05 17:11:54.939769,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
2,204,2024-01-05 17:10:07.228039,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
3,193,2024-01-05 17:09:35.975965,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
4,182,2024-01-05 17:08:08.702625,22.0,1,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,117,2024-01-05 17:01:22.195014,35.99,11,Stradivarius STR baggy jeans in vintage wash,https://www.asos.com/stradivarius/stradivarius...,www.asos.com,https://images.asos-media.com/products/stradiv...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,15
360,106,2024-01-05 16:51:10.174849,35.99,11,Stradivarius STR baggy jeans in vintage wash,https://www.asos.com/stradivarius/stradivarius...,www.asos.com,https://images.asos-media.com/products/stradiv...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,15
361,95,2024-01-05 16:22:19.797968,35.99,11,Stradivarius STR baggy jeans in vintage wash,https://www.asos.com/stradivarius/stradivarius...,www.asos.com,https://images.asos-media.com/products/stradiv...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,15
362,225,2024-01-05 17:19:05.935351,22.5,12,Monki asymmetric ruffle detail midi dress in red,https://www.asos.com/monki/monki-asymmetric-ru...,www.asos.com,https://images.asos-media.com/products/monki-a...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,16


### Finding the most recent price of each subscription for 1 user

In [32]:
sorted_df = user_id_1_df.sort_values(
    by=['Product ID', 'Updated At'], ascending=[True, False])

# Group by 'Product ID' and take the first (most recent) entry of each group
most_recent_prices = sorted_df.groupby('Product ID').first()

most_recent_prices

,Price ID,Updated At,Price,Product Name,Product URL,Website Name,Image URL,Product Availability,User ID,User Email,User FirstName,User LastName,Subscription ID
Product ID,,,,,,,,,,,,,
1,228,2024-01-05 17:19:05.936439,22.0,ASOS DESIGN off shoulder twist midi dress with...,https://www.asos.com/asos-design/asos-design-o...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,1
2,217,2024-01-05 17:19:05.929124,22.0,ASOS DESIGN Hourglass slim mom jeans in washed...,https://www.asos.com/asos-design/asos-design-h...,www.asos.com,https://images.asos-media.com/products/asos-de...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,2
3,218,2024-01-05 17:19:05.929307,24.0,Clinique Almost Lipstick - Pink Honey,https://www.asos.com/clinique/clinique-almost-...,www.asos.com,https://images.asos-media.com/products/cliniqu...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,3
6,221,2024-01-05 17:19:05.931555,80.0,Miss Selfridge Premium embellished star puff s...,https://www.asos.com/miss-selfridge/miss-selfr...,www.asos.com,https://images.asos-media.com/products/miss-se...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,8
7,46,2024-01-05 19:15:06.036824,25.0,River Island zebra print wrap midi dress in li...,https://www.asos.com/river-island/river-island...,www.asos.com,https://images.asos-media.com/products/river-i...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,18
8,223,2024-01-05 17:19:05.934472,33.0,New Look belted formal coat in grey,https://www.asos.com/new-look/new-look-belted-...,www.asos.com,https://images.asos-media.com/products/new-loo...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,13
9,224,2024-01-05 17:19:05.934966,28.0,Elemis Pro-Collagen Green Fig Cleansing Balm 50g,https://www.asos.com/elemis/elemis-pro-collage...,www.asos.com,https://images.asos-media.com/products/elemis-...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,12
10,227,2024-01-05 17:19:05.936091,22.99,Stradivarius tailored straight trouser in dark...,https://www.asos.com/stradivarius/stradivarius...,www.asos.com,https://images.asos-media.com/products/stradiv...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,14
11,229,2024-01-05 17:19:06.811122,35.99,Stradivarius STR baggy jeans in vintage wash,https://www.asos.com/stradivarius/stradivarius...,www.asos.com,https://images.asos-media.com/products/stradiv...,True,1,trainee.abbey.wilkinson@sigmalabs.co.uk,Abbey,Wilkinson,15


### Get the names of all the products a user is subscribed to

In [33]:
user_id_1_df["Product Name"].unique()

array(['ASOS DESIGN off shoulder twist midi dress with long sleeves in black',
       'ASOS DESIGN Hourglass slim mom jeans in washed black',
       'Clinique Almost Lipstick - Pink Honey',
       'Miss Selfridge Premium embellished star puff sleeve mini dress with tie back detail in black - BLACK',
       'River Island zebra print wrap midi dress in light pink',
       'New Look belted formal coat in grey',
       'Elemis Pro-Collagen Green Fig Cleansing Balm 50g',
       'Stradivarius tailored straight trouser in dark grey ',
       'Stradivarius STR baggy jeans in vintage wash',
       'Monki asymmetric ruffle detail midi dress in red',
       'My Accessories London touch screen knitted gloves in black '],
      dtype=object)

### Get the Total number of Products a user is subscribed to

In [ ]:
len(user_id_1_df["Product Name"].unique())